In [33]:
'''
 # @ Author: Mitas Ray
 # @ Create Time: 2022-01-13 17:44:00
 # @ Description: This file implements functions from the pricing module
 # to show how to get the price and yield of a trade.
 '''

import numpy as np
import os
from google.cloud import bigquery

from ficc.utils.auxiliary_functions import sqltodf
from ficc.pricing.auxiliary_functions import transform_reference_data
from ficc.pricing.price import compute_price
from ficc.pricing.yield_rate import compute_yield

In [34]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../ficc/creds.json"    # put the location of your `creds.json` file

### Conditions that we tested on:
* `yield > 0`: conceptually, this notebook should work for any yield, but in practice, bonds with zero yield indicates a problem with MSRB data, and negative yields haven't been tested on
* `is_non_transaction_based_compensation is false`, `is_lop_or_takedown is false`: conceptually, this notebook should work without these filters, but in practice, away from market prices sometimes have anomolous yields reported in the MSRB data
* `callable_at_cav is false`: this notebook will not be correct for bonds callable at cav; future work will incorporate the special case of cav bonds that have call prices from ICE data
* `ice_valid_TO_date > timestamp(publish_datetime, "America/New_York")`, `timestamp(publish_datetime, "America/New_York") >= ice_valid_from_date`, `msrb_valid_to_date > publish_datetime`, `msrb_valid_from_date <= publish_datetime`: conditions to correctly join ICE and MSRB
* `par_call_price = 100`: conceptually this is identical to the fourth bullet point, but sometimes there are issues in the ICE data
* `interest_payment_frequency is not NULL`: if interest payment frequency is NULL, then the coupon frequency is unknown; possibly the field `coupon_type` could be used to determine the interest payment frequency

In [17]:
query = lambda trade_date: ''' SELECT
                                IFNULL(settlement_date, assumed_settlement_date) AS settlement_date,
                                trade_date,
                                cusip,
                                accrual_date,
                                dollar_price,
                                issue_price,
                                coupon,
                                interest_payment_frequency,
                                next_call_date,
                                par_call_date,
                                next_call_price,
                                par_call_price,
                                maturity_date,
                                previous_coupon_payment_date,
                                next_coupon_payment_date,
                                first_coupon_date,
                                coupon_type,
                                muni_security_type,
                                called_redemption_type,
                                refund_date,
                                refund_price,
                                is_callable,
                                is_called,
                                call_timing,
                                yield,
                                called_redemption_date,
                                rtrs_control_number,
                                has_zero_coupons,
                                last_period_accrues_from_date
                            FROM 
                                `eng-reactor-287421.auxiliary_views.trades_with_ref_data_pd`
                            WHERE 
                                yield > 0 and
                                is_non_transaction_based_compensation is false and 
                                callable_at_cav is false and 
                                is_lop_or_takedown is false and 
                                ice_valid_TO_date > timestamp(publish_datetime, "America/New_York") and
                                timestamp(publish_datetime, "America/New_York") >= ice_valid_from_date and
                                msrb_valid_to_date > publish_datetime and
                                msrb_valid_from_date <= publish_datetime and
                                par_call_price = 100 and
                                interest_payment_frequency is not NULL and 
                                trade_date = \'''' + trade_date + '''\'
                            '''

In [37]:
date = '2021-12-01'

In [38]:
bqclient = bigquery.Client()
muni_df = sqltodf(query(date), bqclient)
transform_reference_data(muni_df)

In [39]:
muni_df    # visualize the dataframe

,settlement_date,trade_date,cusip,accrual_date,dollar_price,issue_price,coupon,interest_payment_frequency,next_call_date,par_call_date,...,refund_price,is_callable,is_called,call_timing,yield,called_redemption_date,rtrs_control_number,has_zero_coupons,last_period_accrues_from_date,deferred
0,2021-12-03,2021-12-01,343667YC9,2015-07-01,105.817,93.629000,3.50,2,2025-08-01,2025-08-01,...,NaN,True,False,1.0,1.849,None,2021120102718600,False,2033-02-01,False
1,2021-12-03,2021-12-01,343667YC9,2015-07-01,105.824,93.629000,3.50,2,2025-08-01,2025-08-01,...,NaN,True,False,1.0,1.847,None,2021120102690100,False,2033-02-01,False
2,2021-12-03,2021-12-01,646039VK0,2014-12-18,115.040,116.076161,5.00,2,2025-06-01,2025-06-01,...,100.0,True,False,1.0,0.641,2025-06-01,2021120104418600,False,2024-12-01,False
3,2021-12-03,2021-12-01,7962535R9,2016-07-28,111.944,116.187953,4.00,2,2026-08-01,2026-08-01,...,NaN,True,False,1.0,1.347,None,2021120105700900,False,2031-08-01,False
4,2021-12-03,2021-12-01,613612AN0,2015-06-24,102.206,98.679056,4.25,2,2025-01-15,2025-01-15,...,NaN,True,False,1.0,3.496,None,2021120101194500,False,2028-07-15,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16904,2021-12-03,2021-12-01,839134GF4,2020-07-08,104.451,103.226000,2.00,2,2027-05-01,2027-05-01,...,NaN,True,False,1.0,1.149,None,2021120104164500,False,2029-11-01,False
16905,2021-12-03,2021-12-01,797299GG3,2012-07-03,101.480,96.804944,4.50,2,2022-04-15,2022-04-15,...,100.0,True,False,1.0,0.454,2022-04-15,2021120100005100,False,2021-10-15,False
16906,2021-12-03,2021-12-01,91412G3J2,2017-12-21,122.234,123.370930,5.00,2,2027-05-15,2027-05-15,...,NaN,True,False,1.0,0.820,None,2021120106373600,False,2027-11-15,False
16907,2021-12-03,2021-12-01,91412G3J2,2017-12-21,121.520,123.370930,5.00,2,2027-05-15,2027-05-15,...,NaN,True,False,1.0,0.940,None,2021120105016800,False,2027-11-15,False


Apply the `compute_price` function to every trade

In [40]:
muni_df['price_calc_from_yield'] = muni_df.apply(lambda x: compute_price(x), axis=1)

UnboundLocalError: local variable 'end_date' referenced before assignment

Plot the price from the reference data on the x-axis and our computed price on the y-axis

In [ ]:
muni_df["price_from_yield"] = [x[0] for x in muni_df['price_calc_from_yield']]
muni_df.plot.scatter(x='dollar_price', y='price_from_yield', c='DarkBlue')

Return the sum and mean of the errors where the error is defined as the absolute distance between our computed price and the reference price

In [ ]:
muni_df['price_delta'] = abs(muni_df['price_from_yield'] - muni_df['dollar_price'])
print("Sum of errors: {}".format(np.sum(muni_df['price_delta'])))
print("Mean of errors: {}".format(np.mean(muni_df['price_delta'])))

Apply the `compute_yield` function to every trade

In [ ]:
muni_df["yield_and_calc_date"] = muni_df.apply(lambda x: compute_yield(x),axis=1)

Plot the yield from the reference data on the x-axis and our computed yield on the y-axis

In [ ]:
muni_df["yield_from_price"] = [x[0] for x in muni_df['yield_and_calc_date']]
muni_df["yield_calc_date"] = [x[1] for x in muni_df['yield_and_calc_date']]
# when we get close to redemption, it's difficult to compute the yield from the price
muni_df.plot.scatter(x='yield', y='yield_from_price', c='DarkBlue')

Return the sum and mean of the errors where the error is defined as the absolute distance between our computed yield and the reference yield

In [ ]:
muni_df['ytw_delta'] = abs(muni_df['yield_from_price'] - muni_df['yield'])
print("Sum of errors: {}".format(np.sum(muni_df['ytw_delta'])))
print("Mean of errors: {}".format(np.mean(muni_df['ytw_delta'])))